# ⚠ Warning

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/OpinionatedGeek%2Fmango-explorer/HEAD?filepath=Notification.ipynb) _🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

# 🥭 Notification

This notebook contains code to send arbitrary notifications.


In [ ]:
import abc
import logging
import requests
import typing


# NotificationTarget class

This base class is the root of the different notification mechanisms.

Derived classes should override `send_notification()` to implement their own sending logic.

Derived classes should not override `send()` since that is the interface outside classes call and it's used to ensure `NotificationTarget`s don't throw an exception when sending.

In [ ]:
class NotificationTarget(metaclass=abc.ABCMeta):
    def __init__(self):
        self.logger: logging.Logger = logging.getLogger(self.__class__.__name__)

    def send(self, item: typing.Any) -> None:
        try:
            self.send_notification(item)
        except Exception as exception:
            self.logger.error(f"Error sending {item} - {self} - {exception}")

    @abc.abstractmethod
    def send_notification(self, item: typing.Any) -> None:
        raise NotImplementedError("NotificationTarget.send() is not implemented on the base type.")

    def __repr__(self) -> str:
        return f"{self}"


# TelegramNotificationTarget class

The `TelegramNotificationTarget` sends messages to Telegram.

The format for the telegram notification is:
1. The word 'telegram'
2. A colon ':'
3. The chat ID
4. An '@' symbol
5. The bot token

For example:
```
telegram:<CHAT-ID>@<BOT-TOKEN>
```

The [Telegram instructions to create a bot](https://core.telegram.org/bots#creating-a-new-bot) show you how to create the bot token.

In [ ]:
class TelegramNotificationTarget(NotificationTarget):
    def __init__(self, address):
        super().__init__()
        chat_id, bot_id = address.split("@", 1)
        self.chat_id = chat_id
        self.bot_id = bot_id

    def send_notification(self, item: typing.Any):
        payload = {"disable_notification": True, "chat_id": self.chat_id, "text": str(item)}
        url = f"https://api.telegram.org/bot{self.bot_id}/sendMessage"
        headers = {"Content-Type": "application/json"}
        requests.post(url, json=payload, headers=headers)

    def __str__(self) -> str:
        return f"Telegram chat ID: {self.chat_id}"


# DiscordNotificationTarget class

The `DiscordNotificationTarget` sends messages to Discord.


In [ ]:
class DiscordNotificationTarget(NotificationTarget):
    def __init__(self, address):
        super().__init__()
        self.address = address

    def send_notification(self, item: typing.Any):
        payload = {
            "content": str(item)
        }
        url = self.address
        headers = {"Content-Type": "application/json"}
        requests.post(url, json=payload, headers=headers)

    def __str__(self) -> str:
        return "Discord webhook"


# parse_subscription_target() function

`parse_subscription_target()` takes a parameter as a string and returns a notification target.

This is most likely used when parsing command-line arguments - this function can be used in the `type` parameter of an `add_argument()` call.

In [ ]:
def parse_subscription_target(target):
    protocol, address = target.split(":", 1)

    if protocol == "telegram":
        return TelegramNotificationTarget(address)
    elif protocol == "discord":
        return DiscordNotificationTarget(address)
    else:
        raise Exception(f"Unknown protocol: {protocol}")


# ✅ Testing

In [ ]:
def _notebook_tests():
    test_target = parse_subscription_target("telegram:chat@bot")

    assert(test_target.chat_id == "chat")
    assert(test_target.bot_id == "bot")


_notebook_tests()
del _notebook_tests

# 🏃 Running

A few quick examples to show how to use these functions

In [ ]:
if __name__ == "__main__":
    test_target = parse_subscription_target("telegram:chat@bot")
    print(test_target)
